<a href="https://colab.research.google.com/github/Farhaj499/RAG_with_Weaviate_DB/blob/main/RAG_with_Weaviate_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://console.weaviate.cloud/

In [6]:
!pip install weaviate-client langchain langchain_google_genai

In [7]:
WEAVIATE_API_KEY = "9FdtrS8mShbJ9RgtVbvL0gKUMF0CiZtVA4tE"
WEAVIATE_CLUSTER = "https://ql4sueietoqf9kwgip5jya.c0.asia-southeast1.gcp.weaviate.cloud"

In [8]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key = GOOGLE_API_KEY)

a = llm.invoke("Hi")
print(a.content)

Hi there! How can I help you today?



## Data Reading

In [10]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [11]:
!pip install unstructured
!pip install "unstructured[pdf]"

In [12]:
!pip install -U langchain-community

In [13]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("./data",glob = "**/*.pdf")
data = loader.load()

Pdf used: Agentic AI
https://www.artificiality.world/content/files/2024/05/Agentic-AI.pdf

In [14]:
data

[Document(metadata={'source': 'data/Agentic-AI.pdf'}, page_content='Agentic AI (aka AI Agents)\n\nThe future of AI is agentic.\n\nPublished: May 2024\n\nKey Points:\n\nAgentic AI: Agentic AI systems represent a signiﬁcant advancement in AI, characterized by their ability to\n\nperceive, reason, and act autonomously, extending human capabilities in unprecedented ways.\n\nDeﬁnitions and Perspectives: Various deﬁnitions from organizations like OpenAI, IBM, and Google\n\nDeepMind highlight the multifaceted nature of agentic AI, emphasizing goals, actions, and natural language\n\ninterfaces. At Artiﬁciality, we deﬁne agentic AI systems as those that can perceive, reason, and act with varying\n\ncomplexity to extend the human mind beyond our current experience.\n\nHistorical Context and Development: The concept of AI agency has evolved from early systems like IBM\'s\n\nDeep Blue to advanced platforms such as AutoGPT and LangChain, showcasing rapid advancements in the\n\nﬁeld.\n\nCore Compone

## Text Splitting

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(data)

In [16]:
docs

[Document(metadata={'source': 'data/Agentic-AI.pdf'}, page_content="Agentic AI (aka AI Agents)\n\nThe future of AI is agentic.\n\nPublished: May 2024\n\nKey Points:\n\nAgentic AI: Agentic AI systems represent a signiﬁcant advancement in AI, characterized by their ability to\n\nperceive, reason, and act autonomously, extending human capabilities in unprecedented ways.\n\nDeﬁnitions and Perspectives: Various deﬁnitions from organizations like OpenAI, IBM, and Google\n\nDeepMind highlight the multifaceted nature of agentic AI, emphasizing goals, actions, and natural language\n\ninterfaces. At Artiﬁciality, we deﬁne agentic AI systems as those that can perceive, reason, and act with varying\n\ncomplexity to extend the human mind beyond our current experience.\n\nHistorical Context and Development: The concept of AI agency has evolved from early systems like IBM's\n\nDeep Blue to advanced platforms such as AutoGPT and LangChain, showcasing rapid advancements in the\n\nﬁeld.\n\nCore Componen

In [17]:
len(docs)

49

## Embedding Convertion

In [18]:
!pip install transformers -U

In [20]:
!pip install -qU langchain-huggingface

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
embeddings = embedding

## Vector Database Storage

In [22]:
!pip install -Uqq langchain-weaviate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 9.7 MB/s eta 0:00:00


In [30]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore
from weaviate.classes.init import Auth
import os

#Connect to weaviate Cluster
WEAVIATE_URL = WEAVIATE_CLUSTER

huggingface_key = userdata.get("HF_TOKEN")
client = weaviate.connect_to_weaviate_cloud(
    cluster_url = WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),      # `weaviate_key`: your Weaviate API key
    headers = {"X-HuggingFace-Api-Key": huggingface_key},
)

In [31]:
client.is_ready()

True

In [42]:
db = WeaviateVectorStore.from_documents(docs, embeddings, client=client)

## Similarity Measurement

In [46]:
query = "what is a vertical AI Agent?"

# retrieve text related to the query
docs = db.similarity_search(query, k=2)

In [47]:
docs

[Document(metadata={'source': 'data/Agentic-AI.pdf'}, page_content='Organizational Structures: Single-agent and multi-agent architectures (vertical and horizontal) each have\n\ndistinct advantages and challenges, inﬂuencing scalability, robustness, and ﬂexibility.\n\nArtiﬁciality\n\nPage 1\n\nhello@artiﬁciality.world\n\nIntroduction\n\nRecent developments in AI have given rise to a new class of systems known as agentic AI. These systems are\n\ncharacterized by their ability to perceive, reason, and act with varying levels of complexity, extending human\n\ncapabilities in unprecedented ways. While there isn’t a single deﬁnition of agentic AI—what is agreed upon is that\n\nagentic AI represents a signiﬁcant leap.\n\nOpenAI describes agentic AI systems as those that\n\n"can pursue complex goals with limited direct\n\nsupervision." Researchers from Stanford, Microsoft,\n\nand UCLA characterize AI agents as "a class of\n\ninteractive systems that can perceive visual stimuli,\n\nlanguage inp

In [48]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# The prompt expects input with keys for "context" and "question"
prompt = ChatPromptTemplate.from_template(template)
retriever=db.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke("what is a vertical AI Agent?")

'Based on the provided text, a vertical AI agent is a multi-agent system with a hierarchical structure.  It offers effective delegation, specialization, and effective coordination as advantages.  However, disadvantages include communication bottlenecks, cascading failures, and limited adaptability.\n'